In [19]:
from pylab import *
import os
import sys
from sklearn.mixture import GaussianMixture
import pickle
from PIL import Image

In [79]:
dataset_dir = 'rgbd_dataset_freiburg3_long_office_household'
depth_dir = 'depth'
gmm_dir = 'gmm_qqvga'

# og 
fx = 525.0  # focal length x
fy = 525.0  # focal length y
cx = 319.5  # optical center x
cy = 239.5  # optical center y

# fri3 
fx = 535.4  # focal length x
fy = 539.2  # focal length y
cx = 320.1  # optical center x
cy = 247.6  # optical center y
factor = 5000 # for the 16-bit PNG files
# OR: factor = 1 # for the 32-bit float images in the ROS bag files

full_depth_dir = os.path.join(dataset_dir,depth_dir)
full_gmm_dir = os.path.join(dataset_dir,gmm_dir)
if not os.path.exists(full_gmm_dir):
    os.mkdir(full_gmm_dir)

In [80]:
y_grid = np.repeat(np.arange(480)[:,None],640,1)
x_grid = np.repeat(np.arange(640)[None,:],480,0)

def img_to_pts(img):
    Z = img/factor
    X = (x_grid-cx) * Z /fx
    Y = (y_grid-cy) * Z /fy
    #for v in range(depth_image.height):
    #  for u in range(depth_image.width):
    #    Z = depth_image[v,u] / factor;
    #    X = (u - cx) * Z / fx;
    #    Y = (v - cy) * Z / fy;
    X = X[::4,::4]
    Y = Y[::4,::4]
    Z = Z[::4,::4]
    print(X.shape)

    xl = X[Z > 0]
    yl = Y[Z > 0]
    zl = Z[Z > 0]
    n = zl.shape[0]
    print(n)
    idx = np.random.randint(n, size=n//16)

    return np.vstack([xl,yl,zl]).T

def load_data(file):
    image = Image.open(file)   
    pixel = np.array(image)
    return img_to_pts(pixel)

for fl in sorted(os.listdir(full_depth_dir)):
    name,ext = os.path.splitext(fl)
    new_name = name + '.pkl'
    
    clf = GaussianMixture(100)
    data = load_data(os.path.join(full_depth_dir,fl))
    t1 = time.time()
    clf.fit(data)
    print(time.time()-t1)
    with open(os.path.join(full_gmm_dir,new_name),'wb') as fp:
        pickle.dump(clf,fp)
    

(120, 160)
15493
9.730216026306152
(120, 160)
15925
7.803688049316406
(120, 160)
15860
9.94141697883606
(120, 160)
15877
7.254570007324219
(120, 160)
15731
8.327094078063965
(120, 160)
15840
10.012603998184204
(120, 160)
15824
8.554364204406738
(120, 160)
15816
11.418878078460693
(120, 160)
15897
6.755975008010864
(120, 160)
15898
9.33423900604248
(120, 160)
15835
9.46579098701477
(120, 160)
15796
8.52143406867981
(120, 160)
15822
12.150306940078735
(120, 160)
15944
9.669536828994751
(120, 160)
15873
8.663399934768677
(120, 160)
15879
8.819669008255005
(120, 160)
15850
8.639222860336304
(120, 160)
15877
11.486099004745483
(120, 160)
15875
8.681761026382446
(120, 160)
15981
8.460163116455078
(120, 160)
15813
9.658653974533081
(120, 160)
15839
7.840821981430054
(120, 160)
15836
10.17200517654419
(120, 160)
15866
10.724243879318237
(120, 160)
15878
8.579099893569946
(120, 160)
15889
10.290215969085693
(120, 160)
15778
11.222356081008911
(120, 160)
15764
8.062595129013062
(120, 160)
15808


13.054299116134644
(120, 160)
16419
9.797453880310059
(120, 160)
16423
13.33731722831726
(120, 160)
16483
13.622584104537964
(120, 160)
16471
12.195724964141846
(120, 160)
16368
9.315844058990479
(120, 160)
16552
10.321016073226929
(120, 160)
16541
8.85027289390564
(120, 160)
16545
13.846367120742798
(120, 160)
16483
8.456857919692993
(120, 160)
16660
8.229511976242065
(120, 160)
16612
11.714063167572021
(120, 160)
16662
11.80259919166565
(120, 160)
16731
12.171727657318115
(120, 160)
16699
17.36001682281494
(120, 160)
16706
12.92510199546814
(120, 160)
16670
13.68186092376709
(120, 160)
16621
11.391003847122192
(120, 160)
16727
12.336008071899414
(120, 160)
16683
8.47184705734253
(120, 160)
16770
13.485177040100098
(120, 160)
16645
12.077792167663574
(120, 160)
16634
10.12535810470581
(120, 160)
16636
8.596765995025635
(120, 160)
16590
11.979410886764526
(120, 160)
16608
11.792587995529175
(120, 160)
16732
11.088985919952393
(120, 160)
16666
11.559781789779663
(120, 160)
16671
11.7860

11.322279214859009
(120, 160)
15683
12.883458137512207
(120, 160)
15556
12.493226051330566
(120, 160)
15601
11.308650016784668
(120, 160)
15654
8.79113483428955
(120, 160)
15738
8.034300088882446
(120, 160)
15680
12.103539943695068
(120, 160)
15767
10.034878253936768
(120, 160)
15729
9.900816917419434
(120, 160)
15752
8.46844220161438
(120, 160)
15762
9.004632711410522
(120, 160)
15833
8.263842821121216
(120, 160)
15882
11.706782102584839
(120, 160)
15848
9.427531003952026
(120, 160)
15762
7.905694961547852
(120, 160)
15788
7.154929876327515
(120, 160)
15716
9.522957801818848
(120, 160)
15767
13.305477142333984
(120, 160)
15696
8.91968321800232
(120, 160)
15774
9.682500123977661
(120, 160)
15914
10.918078899383545
(120, 160)
15968
9.963140964508057
(120, 160)
15798
12.648027896881104
(120, 160)
15809
12.565099000930786
(120, 160)
15930
10.02859878540039
(120, 160)
15938
13.051201105117798
(120, 160)
15902
12.109117031097412
(120, 160)
15839
13.47889518737793
(120, 160)
15736
8.14124894

10.578228235244751
(120, 160)
16012
10.050355911254883
(120, 160)
16058
9.877971172332764
(120, 160)
16080
9.0935800075531
(120, 160)
16050
11.230223178863525
(120, 160)
16187
9.968365907669067
(120, 160)
16170
10.233814001083374
(120, 160)
16086
10.051037073135376
(120, 160)
16102
9.192621231079102
(120, 160)
16107
10.899074792861938
(120, 160)
16091
11.372445344924927
(120, 160)
16023
9.603152990341187
(120, 160)
15792
7.202810049057007
(120, 160)
15835
6.207281112670898
(120, 160)
16017
9.91375994682312
(120, 160)
15982
10.747753858566284
(120, 160)
16001
6.742946624755859
(120, 160)
16008
11.20174503326416
(120, 160)
16002
9.800993919372559
(120, 160)
15954
8.338312149047852
(120, 160)
15953
10.513361930847168
(120, 160)
15164
7.926301956176758
(120, 160)
14902
7.847073078155518
(120, 160)
15863
8.501442909240723
(120, 160)
15843
12.52907109260559
(120, 160)
15848
9.41654109954834
(120, 160)
15938
7.8713390827178955
(120, 160)
15862
9.567079782485962
(120, 160)
15911
9.359732151031

7.2840070724487305
(120, 160)
16631
10.428349018096924
(120, 160)
16618
8.582555055618286
(120, 160)
16608
10.360047101974487
(120, 160)
16596
10.152023792266846
(120, 160)
16598
10.316543102264404
(120, 160)
16579
8.25226902961731
(120, 160)
16605
8.929372310638428
(120, 160)
16619
9.843179941177368
(120, 160)
16594
9.864654064178467
(120, 160)
16549
9.506873846054077
(120, 160)
16517
9.14796495437622
(120, 160)
16427
11.706638813018799
(120, 160)
16436
10.323160171508789
(120, 160)
16427
8.515393018722534
(120, 160)
16429
8.314949035644531
(120, 160)
16430
7.902331829071045
(120, 160)
16532
7.312334060668945
(120, 160)
16572
7.1113669872283936
(120, 160)
16594
7.383203744888306
(120, 160)
16641
9.527102947235107
(120, 160)
16664
7.39636492729187
(120, 160)
16657
8.061876058578491
(120, 160)
16645
8.956905841827393
(120, 160)
16682
8.632716655731201
(120, 160)
16657
6.7174201011657715
(120, 160)
16610
7.371419906616211
(120, 160)
16636
9.305416107177734
(120, 160)
16645
7.988227844238

...

11.000340938568115
(120, 160)
14918
10.457457065582275
(120, 160)
15025
13.009997844696045
(120, 160)
14898
10.28973388671875
(120, 160)
14634
9.788404941558838
(120, 160)
15066
8.489283800125122
(120, 160)
15431
13.17863392829895
(120, 160)
15515
8.467310905456543
(120, 160)
15361
11.665803909301758
(120, 160)
15325
8.76016902923584
(120, 160)
15311
8.630620002746582
(120, 160)
15258
10.769793033599854
(120, 160)
15421
12.822693824768066
(120, 160)
15465
9.093472003936768
(120, 160)
15352
11.429678678512573
(120, 160)
15459
9.527314186096191
(120, 160)
15291
12.508525848388672
(120, 160)
15274
9.559535026550293
(120, 160)
15556
8.287484884262085
(120, 160)
15334
13.044856071472168
(120, 160)
15298
9.74920105934143
(120, 160)
15494
16.916560888290405
(120, 160)
15376
13.843938827514648
(120, 160)
15393
13.92540168762207
(120, 160)
15512
12.31339406967163
(120, 160)
15357
13.812749147415161
(120, 160)
15280
7.8788769245147705
(120, 160)
15439
8.54653286933899
(120, 160)
15272
9.77577781

10.759016036987305
(120, 160)
15724
10.83597707748413
(120, 160)
15898
9.812854051589966
(120, 160)
15777
11.3944730758667
(120, 160)
15524
11.593952894210815
(120, 160)
15655
9.541422843933105
(120, 160)
15651
9.472440958023071
(120, 160)
15430
11.491762161254883
(120, 160)
15542
12.232921838760376
(120, 160)
15338
10.280245065689087
(120, 160)
15440
7.178091049194336
(120, 160)
15358
7.51504373550415
(120, 160)
14976
12.153881788253784
(120, 160)
15337
9.137945890426636
(120, 160)
14692
12.206131935119629
(120, 160)
15315
8.491931200027466
(120, 160)
15540
8.450175046920776
(120, 160)
15720
9.775309085845947
(120, 160)
15751
9.86033821105957
(120, 160)
15811
9.789405345916748
(120, 160)
15793
11.882817029953003
(120, 160)
15723
11.44754409790039
(120, 160)
15823
8.83096194267273
(120, 160)
16034
10.476794719696045
(120, 160)
15956
12.591584920883179
(120, 160)
15843
8.404019117355347
(120, 160)
15996
11.664085149765015
(120, 160)
15917
12.607081413269043
(120, 160)
15743
9.3611052036

9.01432180404663
(120, 160)
16876
7.958096742630005
(120, 160)
16865
8.143379926681519
(120, 160)
16897
6.93198299407959
(120, 160)
16918
7.531796216964722
(120, 160)
16879
6.710627794265747
(120, 160)
16892
6.796985149383545
(120, 160)
16798
10.686622381210327
(120, 160)
16776
9.200850009918213
(120, 160)
16907
9.076836824417114
(120, 160)
16913
7.766009092330933
(120, 160)
16931
11.021811246871948
(120, 160)
16935
7.721066951751709
(120, 160)
16899
11.234133005142212
(120, 160)
16900
7.282236814498901
(120, 160)
16911
9.954111337661743
(120, 160)
16944
9.603134870529175
(120, 160)
16933
10.060857057571411
(120, 160)
16906
11.561092853546143
(120, 160)
16906
8.7262442111969
(120, 160)
16868
9.046730995178223
(120, 160)
16833
9.296359300613403
(120, 160)
16832
8.396177053451538
(120, 160)
16813
9.682297945022583
(120, 160)
16863
6.960919141769409
(120, 160)
16857
9.741931915283203
(120, 160)
16845
7.797522068023682
(120, 160)
16819
11.540226936340332
(120, 160)
16821
10.702073097229004

11.4878671169281
(120, 160)
16518
10.629755020141602
(120, 160)
16713
10.062326908111572
(120, 160)
16745
10.58278489112854
(120, 160)
16701
8.102702856063843
(120, 160)
16772
7.909861087799072
(120, 160)
16752
10.215699911117554
(120, 160)
16773
7.804732799530029
(120, 160)
16803
11.783902883529663
(120, 160)
16726
10.451708793640137
(120, 160)
16719
8.838851928710938
(120, 160)
16739
8.782445192337036
(120, 160)
16668
10.472787857055664
(120, 160)
16606
7.4066078662872314
(120, 160)
16620
7.447051048278809
(120, 160)
16577
6.760539770126343
(120, 160)
16552
8.521639823913574
(120, 160)
16508
10.612711906433105
(120, 160)
16517
10.186131000518799
(120, 160)
16573
9.441660165786743
(120, 160)
16584
10.757937908172607
(120, 160)
16599
7.522013902664185
(120, 160)
16458
8.481933116912842
(120, 160)
16496
6.809009790420532
(120, 160)
16535
10.130713939666748
(120, 160)
16520
8.859013795852661
(120, 160)
16489
7.8988401889801025
(120, 160)
16377
7.888029098510742
(120, 160)
16536
8.3001110

11.194445133209229
(120, 160)
15843
17.111270904541016
(120, 160)
15875
13.170756101608276
(120, 160)
15998
14.314486980438232
(120, 160)
15813
10.909708023071289
(120, 160)
15770
12.558485984802246
(120, 160)
15862
13.318869829177856
(120, 160)
15814
15.333483934402466
(120, 160)
15715
9.559202194213867
(120, 160)
15831
9.96454906463623
(120, 160)
15789
11.194396018981934
(120, 160)
15728
19.29823112487793
(120, 160)
15843
11.733994960784912
(120, 160)
15811
12.74737811088562
(120, 160)
15790
11.07980990409851
(120, 160)
15615
14.369146823883057
(120, 160)
15583
14.036160945892334
(120, 160)
15802
12.854914665222168
(120, 160)
15629
8.780114889144897
(120, 160)
15797
12.971743106842041
(120, 160)
15762
10.50072693824768
(120, 160)
15947
13.619631290435791
(120, 160)
15961
13.296717882156372
(120, 160)
15887
12.279301881790161
(120, 160)
16030
12.603475093841553
(120, 160)
15850
10.288755893707275
(120, 160)
15982
9.399104118347168
(120, 160)
16026
13.533487796783447
(120, 160)
15919
1

In [81]:
name,ext

('1341848067.862836', '.png')